In [2]:
import pandas as pd
import numpy as np


In [ ]:
# Data

df = pd.read_excel("../data/ltcm_exhibits_data.xlsx", sheet_name= 'Exhibit 2', skiprows=2)
df = df.dropna(how='any')
df = df.set_index('Unnamed: 0')
df.index = pd.to_datetime(df.index)
df.index.name = 'Date'
df.head()


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01,1.1,-0.011,-0.013,0.99
1994-04-01,1.1,0.014,0.008,1.00
1994-05-01,1.2,0.068,0.053,1.05
1994-06-01,1.2,-0.039,-0.029,1.02
1994-07-01,1.4,0.116,0.084,1.10


In [11]:
spy_df = pd.read_excel("../data/spy_data.xlsx", sheet_name= 'excess returns')
spy_df = spy_df.dropna(how='any')
spy_df = spy_df.set_index('date')
spy_df.index = pd.to_datetime(spy_df.index)
spy_df.index.name = 'Date'
spy_df.head()

,SPY
Date,
1994-01-31,0.032401
1994-02-28,-0.031964
1994-03-31,-0.050288
1994-04-30,0.007996
1994-05-31,0.012464


In [ ]:
# 2.1

ltcm_gross = df["Gross Monthly Performancea"]  
ltcm_net   = df["Net Monthly Performanceb"] 
spy_excess = spy_df["SPY"]

In [14]:
import statsmodels.api as sm